We compute the cyclotomic hashes from the proof of Corollary 4.5.

In [1]:
load("utils.sage")

In [4]:
def minimal_level(a, with_zeta=False):
    """
    INPUT:
        - A cyclotomic integer.
        - TYPE: Same type as that of an element in
          `CyclotomicField(n).ring_of_integers()`.
    OUTPUT:
        - The minimal level of the input.
        - If `with_zeta` is set, also return a root of unity (in the parent of the input)
        - TYPE: Integer.

    AUTHOR: Toinou, Kiran
    """
    # Create some objects:
    K = a.parent().fraction_field()
    # TODO: What is the cost of the conductor computation?
    N = K.conductor()
    roots = K.roots_of_unity()

    # Captain obvious
    if a in QQ:
        return (1, K(1)) if with_zeta else 1

    # Outer loop: prime factors p of N:
    for p, _ in factor(N):
        d = N//4 if (p == 2 and N%8 == 4) else N//p
        Kd = CyclotomicField(d)
        # Inner loop: roots of unity
        for zeta in roots[:p]:
            # TODO: Check how Sage does this membership test
            if zeta * a in Kd:
                # Reduce the level and recurse
                ans = minimal_level(Kd(zeta * a), with_zeta)
                return (ans[0], K(ans[1])*zeta) if with_zeta else ans
    # If none of the above, then N
    return (N, K(1)) if with_zeta else N

def cyclotomic_hash(a):
    """
    INPUT:
        - A cyclotomic integer.
        - TYPE: Same type as that of an element in
          `CyclotomicField(n).ring_of_integers()`.
    OUTPUT:
        - The minimal level of the input, and the minimal polynomial of a "canonical" representative of the equivalence class.
        - TYPE: Integer; polynomial with integer coefficients.

    AUTHOR: Kiran
    """
    # Force a into minimal level
    N, z0 = minimal_level(a, with_zeta=True)
    K.<z> = CyclotomicField(N)
    b = K(a*z0)
    # Minimize the degree of the minimal polynomial, breaking ties in favor of the lexicographically earlier polynomial
    # (with coefficients read from highest order to lowest order).
    d = None
    roots = K.roots_of_unity()
    for zeta in roots:
        pol = (b*zeta).minpoly()
        if (d is None) or (pol.degree() < d.degree()) or (pol.degree() == d.degree() and list(pol.reverse()) < list(d.reverse())):
            d = pol
    return N, d

def is_cassels_form(a, hash0=None):
    """
    INPUT:
        - A cyclotomic integer.
        - TYPE: Same type as that of an element in
          `CyclotomicField(n).ring_of_integers()`.
    OUTPUT:
        - True if the input appears in one of Cassels's infinite families.
        - TYPE: Boolean.

    AUTHOR: Kiran
    """
    # Test whether a is a sum of at most two roots of unity.
    K = a.parent()
    roots = K.roots_of_unity()
    if a == 0 or a.multiplicative_order() < Infinity or any((a-z).multiplicative_order() < Infinity for z in roots):
        return True
    # Test whether the castle of a has the right form for one of the other Cassels families.
    u = a*a.conjugate()
    P.<z> = K[]
    l = (z^2 + (u-3)*z + 1).roots(K)
    if len(l) == 0:
        return False
    z1 = l[0][0]
    if z1.multiplicative_order() == Infinity:
        return False
    # Test whether a is equivalent to 1 + z2 - 1/z2 for z2 = sqrt(z1).
    hash1 = cyclotomic_hash(a) if hash0 is None else hash0
    _, z2 = z1.is_square(root=True)
    if _ and hash1 == cyclotomic_hash(1+z2-1/z2):
        return True
    # Test whether a is equivalent to (z5+z5^4) + (z5^2+z5^3)*z1.
    K5.<z5> = CyclotomicField(5)
    if hash1[0] % 5 == 0 and hash1 == cyclotomic_hash((z5+z5^4) + (z5^2+z5^3)*z1):
        return True
    return False


 --- Calculation for the specific integer (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)ζ₄ ---



In [2]:
# The integer involves zeta_5 and zeta_4.
# The LCM of 5 and 4 is 20, so we work in the 20th cyclotomic field.
K = CyclotomicField(20)
z = K.gen()

# Express zeta_5 and zeta_4 in terms of the generator z = zeta_20
z5 = z^4  # zeta_20^4 = zeta_5
z4 = z^5  # zeta_20^5 = zeta_4

# Construct the integer 'a'
a = (z5 + z5^4) + (z5^2 + z5^3)*z4

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the result
print(f"The original element is: (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)ζ₄")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")



The original element is: (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)ζ₄
Minimal Level (N): 20
Canonical Minimal Polynomial: x^4 - 2*x^3 + 2*x^2 - 6*x + 9


--- Calculation for the specific integer 1/2*(sqrt(10) + i*sqrt(2)) ---

In [3]:
# The integer involves sqrt(10) and i*sqrt(2).
# This requires sqrt(5) and sqrt(2), so we need zeta_5 and zeta_8.
# The LCM of 5 and 8 is 40, so we work in the 40th cyclotomic field.
K = CyclotomicField(40)
z = K.gen()

# Express zeta_5 and zeta_8 in terms of the generator z = zeta_40
z5 = z^8  # zeta_40^8 = zeta_5
z8 = z^5  # zeta_40^5 = zeta_8

# Construct the necessary components from roots of unity
# Note: i*sqrt(2) = zeta_8 - zeta_8^7
i_sqrt_2 = z8 - z8^7
# Note: sqrt(5) is the Gauss sum for p=5
sqrt_5 = z5 - z5^2 - z5^3 + z5^4
# Note: sqrt(2) = zeta_8 + zeta_8^7
sqrt_2 = z8 + z8^7
sqrt_10 = sqrt_2 * sqrt_5

# Construct the integer 'a'
# The minimal polynomial x^4 - 4*x^2 + 9 = 0 shows this is an algebraic integer.
a = (sqrt_10 + i_sqrt_2) / 2

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the result
print(f"The original element is: 1/2*(sqrt(10) + i*sqrt(2))")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")


The original element is: 1/2*(sqrt(10) + i*sqrt(2))
Minimal Level (N): 20
Canonical Minimal Polynomial: x^4 - 2*x^3 + 2*x^2 - 6*x + 9


--- Calculation for the specific integer (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)ζ₆ ---

In [7]:


# The integer involves zeta_5 and zeta_6.
# The LCM of 5 and 6 is 30, so we work in the 30th cyclotomic field.
K = CyclotomicField(30)
z = K.gen()

# Express zeta_5 and zeta_6 in terms of the generator z = zeta_30
z5 = z^6  # zeta_30^6 = zeta_5
z6 = z^5  # zeta_30^5 = zeta_6

# Construct the integer 'a'
a = (z5 + z5^4) + (z5^2 + z5^3)*z6

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the result
print(f"The original element is: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")

The original element is: -zeta30^5 - zeta30^4 + zeta30
Minimal Level (N): 15
Canonical Minimal Polynomial: x^4 - 3*x^3 + 5*x^2 - 6*x + 4


--- Calculation for the specific integer 1/2*(sqrt(5) + i*sqrt(3)) ---

In [8]:


# The integer involves sqrt(5) (from zeta_5) and i*sqrt(3) (from zeta_6).
# The LCM of 5 and 6 is 30, so we work in the 30th cyclotomic field.
K = CyclotomicField(30)
z = K.gen()

# Express zeta_5 and zeta_6 in terms of the generator z = zeta_30
z5 = z^6  # zeta_30^6 = zeta_5
z6 = z^5  # zeta_30^5 = zeta_6

# We can express the number as a cyclotomic integer:
# 1/2*(sqrt(5) + i*sqrt(3)) = (zeta_5 + zeta_5^4) + zeta_6
a = (z5 + z5^4) + z6

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the result
print(f"The original element is: 1/2*(sqrt(5) + i*sqrt(3))")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")

The original element is: 1/2*(sqrt(5) + i*sqrt(3))
Minimal Level (N): 15
Canonical Minimal Polynomial: x^4 - 3*x^3 + 5*x^2 - 6*x + 4


--- Calculation for the specific integer (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)ζ₁₀ ---

In [9]:
# The integer involves zeta_5 and zeta_10.
# The LCM of 5 and 10 is 10, so we work in the 10th cyclotomic field.
K = CyclotomicField(10)
z = K.gen()

# Express zeta_5 and zeta_10 in terms of the generator z = zeta_10
z5 = z^2  # zeta_10^2 = zeta_5
z10 = z

# Construct the integer 'a'
a = (z5 + z5^4) + (z5^2 + z5^3)*z10

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# --- Directly check if 'a' is a sum of two roots of unity ---
# The logic is: if a = z1 + z2, then a - z1 = z2.
# So we check if (a - z) is a root of unity for any root of unity z in the field.
field_roots = a.parent().roots_of_unity()
is_sum = False
if a == 0 or a.multiplicative_order() < Infinity or any((a-root).multiplicative_order() < Infinity for root in field_roots):
    is_sum = True

# Print the results
print(f"The original element is: (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)ζ₁₀")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")
print(f"Is the element a sum of two roots of unity? {is_sum}")



The original element is: (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)ζ₁₀
Expressed as a cyclotomic integer: -zeta10^3 - 1
Minimal Level (N): 5
Canonical Minimal Polynomial: x^4 - 5*x^3 + 10*x^2 - 10*x + 5
Is the element a sum of two roots of unity? True


In [10]:
# The number involves roots of unity of order 5.
# We work in the 5th cyclotomic field.
K = CyclotomicField(5)
z = K.gen()

# Construct the integer 'a'
a = 1 - z

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 - ζ₅")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")



The original element is: 1 - ζ₅
Expressed as a cyclotomic integer: -zeta5 + 1
Minimal Level (N): 5
Canonical Minimal Polynomial: x^4 - 5*x^3 + 10*x^2 - 10*x + 5


--- Calculation for the specific integer 1 - ζ₂₁ - ζ₂₁¹³ ---

In [11]:
# The integer involves zeta_21.
# We work in the 21st cyclotomic field.
K = CyclotomicField(21)
z = K.gen()

# Construct the integer 'a'
a = 1 - z - z^13

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 - ζ₂₁ - ζ₂₁¹³")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")



The original element is: 1 - ζ₂₁ - ζ₂₁¹³
Minimal Level (N): 21
Canonical Minimal Polynomial: x^6 - 5*x^5 + 13*x^4 - 22*x^3 + 28*x^2 - 21*x + 7


 --- Calculation for the specific integer 2*cos(2*pi/7) + (1+i*sqrt(3))/2 ---

In [12]:
# The number can be expressed using roots of unity:
# 2*cos(2*pi/7) = zeta_7 + zeta_7^6
# (1+i*sqrt(3))/2 = zeta_6
# The LCM of 7 and 6 is 42, so we work in the 42nd cyclotomic field.
K = CyclotomicField(42)
z = K.gen()

# Express zeta_7 and zeta_6 in terms of the generator z = zeta_42
z7 = z^6  # zeta_42^6 = zeta_7
z6 = z^7  # zeta_42^7 = zeta_6

# Construct the integer 'a'
a = (z7 + z7^6) + z6

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 2*cos(2*pi/7) + (1+i*sqrt(3))/2")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")



The original element is: 2*cos(2*pi/7) + (1+i*sqrt(3))/2
Expressed as a cyclotomic integer: -zeta42^8 + zeta42^7 + zeta42^6 + zeta42
Minimal Level (N): 21
Canonical Minimal Polynomial: x^6 - 5*x^5 + 13*x^4 - 22*x^3 + 28*x^2 - 21*x + 7


--- Calculation for the specific integer 1 + ζ₁₁ + ζ₁₁² + ζ₁₁⁴ + ζ₁₁⁷ ---

In [13]:
# The number involves roots of unity of order 11.
# We work in the 11th cyclotomic field.
K = CyclotomicField(11)
z = K.gen()

# Construct the integer 'a'
a = 1 + z + z^2 + z^4 + z^7

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 + ζ₁₁ + ζ₁₁² + ζ₁₁⁴ + ζ₁₁⁷")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")



The original element is: 1 + ζ₁₁ + ζ₁₁² + ζ₁₁⁴ + ζ₁₁⁷
Minimal Level (N): 11
Canonical Minimal Polynomial: x^2 - x + 3


In [14]:
# The number involves i*sqrt(11), which is related to the Gauss sum for p=11.
# The field Q(sqrt(-11)) is a subfield of the 11th cyclotomic field.
# We work in the 11th cyclotomic field.
K = CyclotomicField(11)
z = K.gen()

# Construct the integer 'a' using the quadratic Gauss sum.
# The Gauss sum for p=11 is i*sqrt(11).
i_sqrt_11 = sum(kronecker(k, 11) * z^k for k in range(1, 11))
a = (1 + i_sqrt_11) / 2

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: (1 + i*sqrt(11))/2")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")



The original element is: (1 + i*sqrt(11))/2
Minimal Level (N): 11
Canonical Minimal Polynomial: x^2 - x + 3


 --- Calculation for the specific integer 1 + ζ₁₃ + ζ₁₃³ + ζ₁₃⁹ ---

In [15]:
# The number involves roots of unity of order 13.
# We work in the 13th cyclotomic field.
K = CyclotomicField(13)
z = K.gen()

# Construct the integer 'a'
a = 1 + z + z^3 + z^9

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 + ζ₁₃ + ζ₁₃³ + ζ₁₃⁹")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")

The original element is: 1 + ζ₁₃ + ζ₁₃³ + ζ₁₃⁹
Minimal Level (N): 13
Canonical Minimal Polynomial: x^4 - 3*x^3 + 5*x^2 - 9*x + 9


--- Calculation for the specific integer (3+sqrt(13) + i*sqrt(26-6*sqrt(13)))/4 ---

In [16]:
# 1. Define the number in the field of algebraic numbers (QQbar).
a_radical = (3 + sqrt(QQbar(13)) + I * sqrt(26 - 6*sqrt(QQbar(13)))) / 4

# 2. Find its minimal polynomial over the rational numbers.
min_poly_a = a_radical.minpoly()

# 3. Create a number field from this polynomial.
K.<a> = NumberField(min_poly_a)

# 4. The conductor of this field is the smallest N for which the number
#    can be represented in the N-th cyclotomic field.
#    The Kronecker-Weber theorem ensures this works for abelian number fields.
try:
    N = K.conductor()
    
    # 5. Embed the abstract number 'a' into the concrete cyclotomic field Q(zeta_N).
    #    There are multiple valid embeddings; the hash function will produce the
    #    same canonical result regardless of which one we choose.
    KN.<z> = CyclotomicField(N)
    a_cyclotomic = K.embeddings(KN)[0](a)

    # 6. Now, calculate the hash using the correct cyclotomic representation.
    level, canonical_min_poly = cyclotomic_hash(a_cyclotomic)

    # Print the results
    print(f"The original element is: (3+sqrt(13) + i*sqrt(26-6*sqrt(13)))/4")
    print(f"Its minimal polynomial is: {min_poly_a}")
    print(f"The number is cyclotomic and its conductor is {N}.")
    print(f"Minimal Level (N): {level}")
    print(f"Canonical Minimal Polynomial: {canonical_min_poly}")

except (NotImplementedError, ValueError):
    print(f"The number defined by the minimal polynomial {min_poly_a} is not a cyclotomic integer.")

The original element is: (3+sqrt(13) + i*sqrt(26-6*sqrt(13)))/4
Its minimal polynomial is: x^4 - 3*x^3 + 5*x^2 - 9*x + 9
The number is cyclotomic and its conductor is 13.
Minimal Level (N): 13
Canonical Minimal Polynomial: x^4 - 3*x^3 + 5*x^2 - 9*x + 9


--- Calculation for the specific integer 1 - ζ₃₅ + ζ₃₅⁷ - ζ₃₅¹¹ - ζ₃₅¹⁶ ---


In [17]:
# The integer involves zeta_35.
# We work in the 35th cyclotomic field.
K = CyclotomicField(35)
z = K.gen()

# Construct the integer 'a'
a = 1 - z + z^7 - z^11 - z^16

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 - ζ₃₅ + ζ₃₅⁷ - ζ₃₅¹¹ - ζ₃₅¹⁶")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")



The original element is: 1 - ζ₃₅ + ζ₃₅⁷ - ζ₃₅¹¹ - ζ₃₅¹⁶
Minimal Level (N): 35
Canonical Minimal Polynomial: x^4 + x^2 + 9


--- Calculation for the specific integer 1/2*(sqrt(5) + i*sqrt(7)) ---

In [18]:
# The number involves sqrt(5) (from Q(zeta_5)) and i*sqrt(7) (from Q(zeta_7)).
# The LCM of 5 and 7 is 35, so we work in the 35th cyclotomic field.
K = CyclotomicField(35)
z = K.gen()

# Express zeta_5 and zeta_7 in terms of the generator z = zeta_35
z5 = z^7  # zeta_35^7 = zeta_5
z7 = z^5  # zeta_35^5 = zeta_7

# Construct the integer 'a' using quadratic Gauss sums.
# sqrt(5) = z5 + z5^4 - z5^2 - z5^3
sqrt_5 = sum(kronecker(k, 5) * z5^k for k in range(1, 5))
# i*sqrt(7) = sum of (k/7)*z7^k
i_sqrt_7 = sum(kronecker(k, 7) * z7^k for k in range(1, 7))

a = (sqrt_5 + i_sqrt_7) / 2

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1/2*(sqrt(5) + i*sqrt(7))")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")



The original element is: 1/2*(sqrt(5) + i*sqrt(7))
Expressed as a cyclotomic integer: -zeta35^21 + zeta35^20 - zeta35^14 + zeta35^10 + zeta35^5
Minimal Level (N): 35
Canonical Minimal Polynomial: x^4 + x^2 + 9


--- Calculation for the specific integer 1 + ζ₇ + ζ₇³ ---

In [19]:
# The number involves roots of unity of order 7.
# We work in the 7th cyclotomic field.
K = CyclotomicField(7)
z = K.gen()

# Construct the integer 'a'
a = 1 + z + z^3

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 + ζ₇ + ζ₇³")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")



The original element is: 1 + ζ₇ + ζ₇³
Expressed as a cyclotomic integer: zeta7^3 + zeta7 + 1
Minimal Level (N): 7
Canonical Minimal Polynomial: x^2 - x + 2


--- Calculation for the specific integer 1/2*(1 + i*sqrt(7)) ---

In [20]:
# The number involves i*sqrt(7), which is related to the Gauss sum for p=7.
# The field Q(sqrt(-7)) is a subfield of the 7th cyclotomic field.
# We work in the 7th cyclotomic field.
K = CyclotomicField(7)
z = K.gen()

# Construct the integer 'a' using the quadratic Gauss sum.
i_sqrt_7 = sum(kronecker(k, 7) * z^k for k in range(1, 7))
a = (1 + i_sqrt_7) / 2

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1/2*(1 + i*sqrt(7))")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")


The original element is: 1/2*(1 + i*sqrt(7))
Expressed as a cyclotomic integer: zeta7^4 + zeta7^2 + zeta7 + 1
Minimal Level (N): 7
Canonical Minimal Polynomial: x^2 - x + 2


--- Calculation for the specific integer (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³) ---

In [21]:
# The integer involves zeta_5.
# So we work in the 5th cyclotomic field.
K = CyclotomicField(5)
z5 = K.gen()

# Construct the integer 'a'
a = (z5 + z5^4) + (z5^2 + z5^3)

# Calculate the hash
level, min_poly = cyclotomic_hash(a)

# Print the result
print(f"The original element is: (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {min_poly}")



The original element is: (ζ₅+ζ₅⁴) + (ζ₅²+ζ₅³)
Minimal Level (N): 1
Canonical Minimal Polynomial: x - 1


 --- Calculation for the specific integer 1 + ζ₈ - ζ₈⁻¹ ---

In [22]:
# The number involves roots of unity of order 8.
# We work in the 8th cyclotomic field.
# Note that ζ₈ - ζ₈⁻¹ is equal to i\sqrt{2}

K = CyclotomicField(8)
z = K.gen()

# Construct the integer 'a'
a = 1 + z - z^-1

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 + ζ₈ - ζ₈⁻¹")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")



The original element is: 1 + ζ₈ - ζ₈⁻¹
Expressed as a cyclotomic integer: zeta8^3 + zeta8 + 1
Minimal Level (N): 8
Canonical Minimal Polynomial: x^2 - 2*x + 3


--- Calculation for the specific integer 1 + ζ₁₂ - ζ₁₂⁻¹ ---


In [23]:
# The number involves i, which can be written as ζ₁₂ - ζ₁₂⁻¹.
# So, 1 + i is equivalent to 1 + ζ₁₂ - ζ₁₂⁻¹.
# We work in the 12th cyclotomic field.
K = CyclotomicField(12)
z = K.gen()

# Construct the integer 'a'
a = 1 + z - z^-1

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 + i")
print(f"Which is equivalent to: 1 + ζ₁₂ - ζ₁₂⁻¹")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")

The original element is: 1 + i
Which is equivalent to: 1 + ζ₁₂ - ζ₁₂⁻¹
Expressed as a cyclotomic integer: zeta12^3 + 1
Minimal Level (N): 4
Canonical Minimal Polynomial: x^2 - 2*x + 2


--- Calculation for the specific integer 1 + ζ₂₀ - ζ₂₀⁻¹ ---

In [24]:
# The number involves roots of unity of order 20.
# We work in the 20th cyclotomic field.
K = CyclotomicField(20)
z = K.gen()

# Construct the integer 'a'
a = 1 + z - z^-1

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 + ζ₂₀ - ζ₂₀⁻¹")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")


The original element is: 1 + ζ₂₀ - ζ₂₀⁻¹
Expressed as a cyclotomic integer: zeta20^7 - zeta20^5 + zeta20^3 + 1
Minimal Level (N): 20
Canonical Minimal Polynomial: x^4 - 4*x^3 + 9*x^2 - 10*x + 5


--- Calculation for the specific integer 1 + 1/2*i*(sqrt(5)+1) ---


In [25]:
# The number involves i (from Q(zeta_4)) and sqrt(5) (from Q(zeta_5)).
# The smallest field containing both is the 20th cyclotomic field.
K = CyclotomicField(20)

# Construct the number using its radical form. Sage can automatically
# convert this into an element of the specified cyclotomic field.
a_radical = 1 + I * (sqrt(5)+1)/2
a = K(a_radical)

# Calculate the hash
level, canonical_min_poly = cyclotomic_hash(a)

# Print the results
print(f"The original element is: 1 + 1/2*i*(sqrt(5)+1)")
print(f"Expressed as a cyclotomic integer: {a}")
print(f"Minimal Level (N): {level}")
print(f"Canonical Minimal Polynomial: {canonical_min_poly}")



The original element is: 1 + 1/2*i*(sqrt(5)+1)
Expressed as a cyclotomic integer: zeta20^7 + zeta20^3 + 1
Minimal Level (N): 20
Canonical Minimal Polynomial: x^4 - 4*x^3 + 9*x^2 - 10*x + 5
